In [2]:
from bs4 import BeautifulSoup
import lxml
import csv
from urllib.request import Request, urlopen


def get_content(url):
  req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()

  return webpage
url='https://www.breitbart.com/?fbclid=IwAR2UPAhsOnYtlAKyTISt3C488eyooGnYVr1V5RZPhWGFHU9DVG5rdM_GjLI'
page = get_content(url)
soup = BeautifulSoup(page,'lxml')
#print(soup.prettify())


In [3]:
category = soup.find("ul",id="NavT").find_all('a')

In [4]:
list_url_category = []
c=0
for i in category:
  c+=1
  if (i.get("href")!=None) and (c!=13):
    list_url_category.append(i.get("href"))
for i in list_url_category:
  print(i)

https://www.breitbart.com/politics/
https://www.breitbart.com/entertainment/
https://www.breitbart.com/the-media/
https://www.breitbart.com/economy/
https://www.breitbart.com/europe/
https://www.breitbart.com/border/
https://www.breitbart.com/middle-east/
https://www.breitbart.com/africa/
https://www.breitbart.com/asia/
https://www.breitbart.com/latin-america/
https://www.breitbart.com/world-news/
https://www.breitbart.com/tech/
https://www.breitbart.com/sports/
https://www.breitbart.com/tag/on-the-hill/
/news/
https://www.breitbart.com/tag/b-inspired-news/


In [6]:
import pandas as pd
df = pd.DataFrame()
title = []
href = []
is_sarcastic = []
for path_category in list_url_category:
  flag = False
  url = path_category 
  count=0
  print(url)
  while flag == False :
    try: 
      page = get_content(url)
    except:
      pass
    soup = BeautifulSoup(page,'lxml')
    # print(soup.prettify())
    list_news = soup.find_all("div",class_="tC")
    data_ = []
    date=[]
    for x in list_news:
      data_.extend(x.find_all("h2"))
      date.extend(x.find_all("time"))
    b=[]
    for x in data_:
      b.extend(x.find_all("a"))
    for i in range(len(date)):
      if (int(date[i].get("datetime")[0:4]) <= 2019):
        flag = True 
        break
      if (count==5000):
        flag=True
        break
      is_sarcastic.append(1)
      title.append(b[i].get("title"))
      href.append(b[i].get("href"))
      count+=1
    next_page = soup.find_all("nav",class_ = "pagination")
    x = next_page[0].find_all("a")
    url  = path_category + x[0].get("href")
    # page = get_content(l)
    # soup = BeautifulSoup(page,'lxml')
    # print(soup.prettify())
    
df['is_sarcastic'] = is_sarcastic
df['headline'] = title
df['article_link'] = href
display(df)
print(len(df))

https://www.breitbart.com/politics/
https://www.breitbart.com/entertainment/
https://www.breitbart.com/the-media/
https://www.breitbart.com/economy/
https://www.breitbart.com/europe/
https://www.breitbart.com/border/
https://www.breitbart.com/middle-east/
https://www.breitbart.com/africa/
https://www.breitbart.com/asia/
https://www.breitbart.com/latin-america/
https://www.breitbart.com/world-news/
https://www.breitbart.com/tech/
https://www.breitbart.com/sports/
https://www.breitbart.com/tag/on-the-hill/
/news/
https://www.breitbart.com/tag/b-inspired-news/


,is_sarcastic,headline,article_link
0,1,"Delingpole: Come Back Theresa May, (Almost) Al...",/europe/2021/06/12/delingpole-come-back-theres...
1,1,French Conservative Magazine Takes Legal Actio...,/europe/2021/06/12/french-conservative-magazin...
2,1,Hong Kong Democracy Activist Agnes Chow Releas...,/europe/2021/06/12/hong-kong-democracy-activis...
3,1,Blair Says Vaccinated Should Enjoy ‘Greater Fr...,/europe/2021/06/12/blair-says-vaccinated-shoul...
4,1,"Crenshaw: Omar Grouping U.S., Israel with Terr...",/clips/2021/06/12/crenshaw-omar-grouping-u-s-i...
...,...,...,...
66723,1,None,/health/2021/06/08/watch-family-helps-reunite-...
66724,1,None,/education/2021/06/07/video-12-year-old-boy-gr...
66725,1,None,/health/2021/06/06/watch-rookie-officer-saves-...
66726,1,WATCH - Teen Paralyzed in Football Game Walks ...,/health/2021/06/06/watch-teen-paralyzed-footba...


66728


In [7]:
def line(headline,article_link,is_sarcastic):
    article_link='https://www.breitbart.com'+article_link
    line = '{"article_link": ' + '"' + article_link + '", "headline": "' + headline + '", "is_sarcastic": ' + str(is_sarcastic) + '}\n'
    return line

In [8]:
import sys,os
f=open('breitbart.json','w')
for i in range(len(df)):
  if (df['headline'][i]!=None):
    f.write(line(df['headline'][i],df['article_link'][i],df['is_sarcastic'][i])+'\n')
f.close()

# Báo **cáo**

Với website https://www.breitbart.com/, lấy headline ở các mục lục page sau:
https://www.breitbart.com/politics/
https://www.breitbart.com/entertainment/
https://www.breitbart.com/the-media/
https://www.breitbart.com/economy/
https://www.breitbart.com/europe/
https://www.breitbart.com/border/
https://www.breitbart.com/middle-east/
https://www.breitbart.com/africa/
https://www.breitbart.com/asia/
https://www.breitbart.com/latin-america/
https://www.breitbart.com/world-news/
https://www.breitbart.com/tech/
https://www.breitbart.com/sports/
https://www.breitbart.com/tag/on-the-hill/
/news/
https://www.breitbart.com/tag/b-inspired-news/

Với mỗi mục lục page sẽ tiến hành chạy từng trang 1(giới hạn bài báo cho mỗi mục lục page là 5000 bài):


*   Thu thập headline và link dẫn từng bài báo
*   Lấy ra thời gian đăng bài báo và chỉ chọn những bài báo được đăng trong 2 năm trở lại đây
* Chạy đến cuối trang thì chuyển sang trang báo tiếp theo

